In [178]:
import pandas as pd
import numpy as np
from IPython.display import display, HTML
display(HTML('<style>.container { width:90% !important; }</style>'))
import random
import warnings
from scipy.stats import poisson
from scipy.special import gammaln
from scipy.optimize import minimize
from multiprocessing import Pool
from matplotlib import colors as mcolors
import pylab as pl
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble.gradient_boosting import GradientBoostingRegressor
from IPython.display import Image
import matplotlib.pyplot as plt
from math import sqrt
from scipy import stats
from itertools import product
import datetime

from tensorflow.keras.layers import Dense, Input, Conv1D, MaxPooling1D, Flatten, Dropout, GlobalMaxPooling1D, LSTM
from keras.regularizers import l1, l2
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop, Adam

In [179]:
weekly_income = pd.read_csv("transactions_per_users_daily.csv")
weekly_income.head()

,user_id,transaction_value,transaction_count,days
0,user_356,5.956667,3,2018-01-01
1,user_375,5.333333,3,2018-01-02
2,user_46,16.170000,3,2018-01-02
3,user_472,4.766000,5,2018-01-02
4,user_657,18.666667,3,2018-01-02


In [180]:
weekly_income['days'] = weekly_income['days'].apply(lambda x: datetime.datetime.strptime(str(x)[0:10], '%Y-%m-%d'))
weekly_income['weeks'] = weekly_income['days'].apply(lambda x: x + datetime.timedelta(days=-x.isoweekday() + 1))
weekly_income = weekly_income.groupby(["user_id", "weeks"]).agg({"transaction_value": "sum"}).reset_index()

In [181]:
weekly_income = weekly_income.sort_values(by=["user_id", "weeks"], ascending=True)

In [182]:
weekly_income['order_seq_num'] = weekly_income.sort_values(by=["user_id", "weeks"]).groupby(['user_id']).cumcount()+1
weekly_income = weekly_income.sort_values(by=["user_id", "weeks"])
weekly_income

,user_id,weeks,transaction_value,order_seq_num
0,user_0,2018-01-15,46.490714,1
1,user_0,2018-01-29,173.471000,2
2,user_0,2018-02-19,114.085000,3
3,user_0,2018-02-26,153.651333,4
4,user_0,2018-03-05,10.417778,5
...,...,...,...,...
315294,user_9999,2019-03-18,71.486667,15
315295,user_9999,2019-03-25,21.915000,16
315296,user_9999,2019-04-08,0.090000,17
315297,user_9999,2019-04-15,17.023333,18


In [183]:
weekly_income = pd.merge(weekly_income, 
                          weekly_income.groupby('user_id'
                                            )['order_seq_num'].max().reset_index().rename(columns={"order_seq_num": "max_order"}), 
                          on='user_id', how='left')
weekly_income['prev_orders'] = weekly_income['max_order'] - 30
weekly_income

,user_id,weeks,transaction_value,order_seq_num,max_order,prev_orders
0,user_0,2018-01-15,46.490714,1,46,16
1,user_0,2018-01-29,173.471000,2,46,16
2,user_0,2018-02-19,114.085000,3,46,16
3,user_0,2018-02-26,153.651333,4,46,16
4,user_0,2018-03-05,10.417778,5,46,16
...,...,...,...,...,...,...
315294,user_9999,2019-03-18,71.486667,15,19,-11
315295,user_9999,2019-03-25,21.915000,16,19,-11
315296,user_9999,2019-04-08,0.090000,17,19,-11
315297,user_9999,2019-04-15,17.023333,18,19,-11


In [184]:
weekly_income = weekly_income.query("order_seq_num > prev_orders")

In [185]:
weekly_income['order_seq_num'] = weekly_income.sort_values(by=["user_id", "weeks"]).groupby(['user_id']).cumcount()+1
weekly_income['order_seq_num'] = weekly_income.apply(
    lambda row: row['order_seq_num'] + abs(row['prev_orders']) if row['prev_orders'] != 0 else row['order_seq_num'], axis=1)
weekly_income

/Users/mac/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/mac/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,user_id,weeks,transaction_value,order_seq_num,max_order,prev_orders
16,user_0,2018-07-09,13.886667,17,46,16
17,user_0,2018-07-16,164.162667,18,46,16
18,user_0,2018-07-23,27.870000,19,46,16
19,user_0,2018-07-30,238.601000,20,46,16
20,user_0,2018-08-06,36.420000,21,46,16
...,...,...,...,...,...,...
315294,user_9999,2019-03-18,71.486667,26,19,-11
315295,user_9999,2019-03-25,21.915000,27,19,-11
315296,user_9999,2019-04-08,0.090000,28,19,-11
315297,user_9999,2019-04-15,17.023333,29,19,-11


In [186]:
def min_max_norm(value, _min, _max):
    if abs(_max - _min) != 0:
        return (value - _min) / abs(_max - _min)
    else: return 0

In [187]:
weekly_income['user_max'], weekly_income['user_min'] = weekly_income['transaction_value'], weekly_income['transaction_value']
users_min_max = weekly_income.groupby("user_id").agg({"user_max": "max", "user_min": "min"}).reset_index()
display(users_min_max.head())
weekly_income = pd.merge(weekly_income.drop(["user_max", "user_min"], axis=1), users_min_max, on='user_id', how='left')
weekly_income.head()

/Users/mac/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,user_id,user_max,user_min
0,user_0,334.010004,1.590
1,user_1,475.000000,0.800
2,user_10,1348.580833,15.897
3,user_100,1537.900000,0.550
4,user_1000,1950.460000,10.000


,user_id,weeks,transaction_value,order_seq_num,max_order,prev_orders,user_max,user_min
0,user_0,2018-07-09,13.886667,17,46,16,334.010004,1.59
1,user_0,2018-07-16,164.162667,18,46,16,334.010004,1.59
2,user_0,2018-07-23,27.870000,19,46,16,334.010004,1.59
3,user_0,2018-07-30,238.601000,20,46,16,334.010004,1.59
4,user_0,2018-08-06,36.420000,21,46,16,334.010004,1.59


In [188]:
weekly_income['transaction_value_norm'] = weekly_income.apply(lambda row: min_max_norm(row['transaction_value'],   
                                                                                         row['user_min'], 
                                                                                         row['user_max']), axis=1)

In [189]:
max_order = max(weekly_income['max_order'])
max_order

71

In [190]:
weekly_income['order_seq_num'] = weekly_income.sort_values(by=["user_id", "weeks"], ascending=False).groupby(['user_id']).cumcount()+0
weekly_income['max_order'] = max_order
weekly_income['order_seq_num'] = weekly_income['max_order'] - weekly_income['order_seq_num']
weekly_income.query("user_id == 'user_0'")

,user_id,weeks,transaction_value,order_seq_num,max_order,prev_orders,user_max,user_min,transaction_value_norm
0,user_0,2018-07-09,13.886667,42,71,16,334.010004,1.59,0.036991
1,user_0,2018-07-16,164.162667,43,71,16,334.010004,1.59,0.489058
2,user_0,2018-07-23,27.870000,44,71,16,334.010004,1.59,0.079057
3,user_0,2018-07-30,238.601000,45,71,16,334.010004,1.59,0.712987
4,user_0,2018-08-06,36.420000,46,71,16,334.010004,1.59,0.104777
5,user_0,2018-09-03,73.767500,47,71,16,334.010004,1.59,0.217127
6,user_0,2018-10-08,63.997000,48,71,16,334.010004,1.59,0.187735
7,user_0,2018-10-15,50.631667,49,71,16,334.010004,1.59,0.147529
8,user_0,2018-10-22,30.240000,50,71,16,334.010004,1.59,0.086186
9,user_0,2018-10-29,13.300000,51,71,16,334.010004,1.59,0.035227


### CNN Model implementation

In [191]:
## tunned paramters:
params = {
    'activation': 'tanh',
    'batch_count': 4,
    'batch_size': 29,
    'epochs': 100,
    'l1': 0.001,
    'l2': 0.001,
    'lag': 2,
    'lahead': 2,
    'lr': 0.001,
    'split_ratio': 0.8,
    'tsteps': 2,
    'units': 2,
    'drop_out_ratio': 0.1
}

In [109]:
weekly_income

,user_id,weeks,transaction_value,order_seq_num,max_order,prev_orders,user_max,user_min,transaction_value_norm
0,user_0,2018-07-09,13.886667,42,71,16,334.010004,1.59,0.036991
1,user_0,2018-07-16,164.162667,43,71,16,334.010004,1.59,0.489058
2,user_0,2018-07-23,27.870000,44,71,16,334.010004,1.59,0.079057
3,user_0,2018-07-30,238.601000,45,71,16,334.010004,1.59,0.712987
4,user_0,2018-08-06,36.420000,46,71,16,334.010004,1.59,0.104777
...,...,...,...,...,...,...,...,...,...
277874,user_9999,2019-03-18,71.486667,67,71,-11,71.486667,0.09,1.000000
277875,user_9999,2019-03-25,21.915000,68,71,-11,71.486667,0.09,0.305687
277876,user_9999,2019-04-08,0.090000,69,71,-11,71.486667,0.09,0.000000
277877,user_9999,2019-04-15,17.023333,70,71,-11,71.486667,0.09,0.237173


In [192]:
monthly_income_per_user = pd.DataFrame(np.array(weekly_income.pivot_table(index="user_id", 
                                                                          columns="order_seq_num", 
                                                                          aggfunc={"transaction_value_norm": "first"}).reset_index()))
monthly_income_per_user = monthly_income_per_user.fillna(0)
monthly_income_per_user.head()

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,user_0,0.036991,0.489058,0.079057,0.712987,0.104777,0.217127,0.187735,0.147529,0.086186,...,0.067134,0.012574,0.319365,0.014364,0.006979,0.000000,0.344446,1.000000,0.136214,0.677449
1,user_1,0.000000,0.000000,0.040489,0.239213,0.105283,0.703729,0.265429,0.484029,0.208140,...,0.119055,0.094264,0.019401,0.029523,0.187453,0.000000,1.000000,0.072121,0.480223,0.021826
2,user_10,0.048114,0.080794,0.217862,0.601820,0.000000,0.034936,1.000000,0.095036,0.066250,...,0.193092,0.021275,0.257200,0.403631,0.119683,0.022831,0.005315,0.138356,0.084608,0.064432
3,user_100,0.264826,0.045426,0.005009,0.136179,0.688249,0.014252,0.000000,0.356258,1.000000,...,0.093479,0.357498,0.025642,0.168836,0.186765,0.006628,0.052636,0.086610,0.053512,0.077222
4,user_1000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.037964,0.818332,0.499443,0.502953,0.502515,1.000000,0.502001,0.503468,0.000000


In [193]:
index = range(len(monthly_income_per_user))
train_index = random.sample(index, int(len(monthly_income_per_user) * params['split_ratio']))
test_index = list(set(index) - set(train_index))

In [194]:
len(monthly_income_per_user[list(monthly_income_per_user.columns)[1:-1]].iloc[train_index])

15012

In [113]:
train = monthly_income_per_user[list(monthly_income_per_user.columns)[1:-1]].iloc[train_index]
test = monthly_income_per_user[list(monthly_income_per_user.columns)[1:-1]].iloc[test_index]

In [114]:
train_x = train.values.reshape(len(train_index), 29, 1)
test_x = test.values.reshape(len(test_index), 29, 1)

In [115]:
train_x.shape[1], train_x.shape[2]

(29, 1)

In [116]:
_input = Input(shape=(29, 1, ))
conv = Conv1D(filters=2, kernel_size=4, padding='same', activation='relu')(_input)
conv = MaxPooling1D(1)(conv)
conv = Dropout(params['drop_out_ratio'])(conv)
conv = LSTM(29, use_bias=False, recurrent_activation=params['activation'],)(conv)
conv = Flatten()(conv)
conv = Dense(8, activation='relu')(conv) 
output = Dense(1, activation='relu')(conv)
model = Model(inputs=_input, outputs=output)
model.compile(loss='mae', optimizer=Adam(lr=params['lr']), metrics=['mae'])
model.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 29, 1)]           0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 29, 2)             10        
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 29, 2)             0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 29, 2)             0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 29)                3596      
_________________________________________________________________
flatten_4 (Flatten)          (None, 29)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 8)                 240 

In [117]:
train_x.shape

(15012, 29, 1)

In [118]:
history = model.fit(train_x,
                    monthly_income_per_user[list(monthly_income_per_user.columns)[-1]].iloc[train_index].values, 
                    batch_size=32,
                    epochs=params['epochs'],
                    verbose=1,
                    validation_split=1-params['split_ratio'],
                    shuffle=True)

Epoch 1/100
376/376 [==============================] - 7s 18ms/step - loss: 0.2192 - mae: 0.2192 - val_loss: 0.2158 - val_mae: 0.2158
Epoch 2/100
376/376 [==============================] - 6s 16ms/step - loss: 0.2178 - mae: 0.2178 - val_loss: 0.2162 - val_mae: 0.2162
Epoch 3/100
376/376 [==============================] - 6s 16ms/step - loss: 0.2174 - mae: 0.2174 - val_loss: 0.2158 - val_mae: 0.2158
Epoch 4/100
376/376 [==============================] - 6s 16ms/step - loss: 0.2163 - mae: 0.2163 - val_loss: 0.2146 - val_mae: 0.2146
Epoch 5/100
376/376 [==============================] - 6s 16ms/step - loss: 0.2158 - mae: 0.2158 - val_loss: 0.2137 - val_mae: 0.2137
Epoch 6/100
376/376 [==============================] - 6s 16ms/step - loss: 0.2154 - mae: 0.2154 - val_loss: 0.2134 - val_mae: 0.2134
Epoch 7/100
376/376 [==============================] - 6s 16ms/step - loss: 0.2144 - mae: 0.2144 - val_loss: 0.2126 - val_mae: 0.2126
Epoch 8/100
376/376 [==============================] - 6s 16ms

Epoch 62/100
376/376 [==============================] - 6s 16ms/step - loss: 0.2081 - mae: 0.2081 - val_loss: 0.2070 - val_mae: 0.2070
Epoch 63/100
376/376 [==============================] - 7s 17ms/step - loss: 0.2079 - mae: 0.2079 - val_loss: 0.2069 - val_mae: 0.2069
Epoch 64/100
376/376 [==============================] - 8s 22ms/step - loss: 0.2079 - mae: 0.2079 - val_loss: 0.2074 - val_mae: 0.2074
Epoch 65/100
376/376 [==============================] - 7s 17ms/step - loss: 0.2077 - mae: 0.2077 - val_loss: 0.2074 - val_mae: 0.2074
Epoch 66/100
376/376 [==============================] - 6s 16ms/step - loss: 0.2080 - mae: 0.2080 - val_loss: 0.2083 - val_mae: 0.2083
Epoch 67/100
376/376 [==============================] - 7s 19ms/step - loss: 0.2076 - mae: 0.2076 - val_loss: 0.2080 - val_mae: 0.2080
Epoch 68/100
376/376 [==============================] - 8s 20ms/step - loss: 0.2078 - mae: 0.2078 - val_loss: 0.2077 - val_mae: 0.2077
Epoch 69/100
376/376 [==============================] -

In [85]:
prediction = model.predict(test_x)
prediction = [i[0] for i in prediction],
prediction

([0.7435221,
  1.0380869,
  0.4999051,
  0.10558546,
  0.0,
  0.08083873,
  0.12766817,
  0.0655996,
  0.89462394,
  0.059293084,
  0.4816339,
  0.6689587,
  0.40031326,
  0.0,
  0.5059254,
  0.5456603,
  0.0,
  0.55340904,
  0.8930368,
  0.11878086,
  1.032813,
  1.0388534,
  0.42270383,
  0.07534278,
  0.114130534,
  0.32268387,
  1.0198824,
  0.54785466,
  0.73888254,
  0.05671998,
  0.382139,
  0.040204436,
  0.22432235,
  0.28912574,
  0.28912574,
  0.03732297,
  0.24155128,
  0.0,
  0.09111779,
  0.027868494,
  0.0,
  0.56325394,
  0.032311752,
  0.0894183,
  0.07526479,
  0.5819565,
  0.03524498,
  0.052858017,
  0.9324601,
  0.5932759,
  0.10209701,
  0.0,
  0.0,
  0.0,
  0.06550356,
  0.10148247,
  0.074556716,
  0.57820094,
  0.0,
  0.28912574,
  0.025134742,
  0.09552961,
  0.29451066,
  1.0107119,
  0.0,
  0.0,
  0.26959148,
  0.1352905,
  0.9940389,
  0.5878116,
  0.06622862,
  0.14897093,
  0.047481902,
  0.717609,
  0.030159973,
  0.6569805,
  0.12385332,
  0.14970434,
 

In [86]:
len(list(monthly_income_per_user[list(monthly_income_per_user.columns)[0]].iloc[test_index]))

3754

In [87]:
len(prediction), len(list(monthly_income_per_user[list(monthly_income_per_user.columns)[0]].iloc[test_index]))

(1, 3754)

In [88]:
prediction

([0.7435221,
  1.0380869,
  0.4999051,
  0.10558546,
  0.0,
  0.08083873,
  0.12766817,
  0.0655996,
  0.89462394,
  0.059293084,
  0.4816339,
  0.6689587,
  0.40031326,
  0.0,
  0.5059254,
  0.5456603,
  0.0,
  0.55340904,
  0.8930368,
  0.11878086,
  1.032813,
  1.0388534,
  0.42270383,
  0.07534278,
  0.114130534,
  0.32268387,
  1.0198824,
  0.54785466,
  0.73888254,
  0.05671998,
  0.382139,
  0.040204436,
  0.22432235,
  0.28912574,
  0.28912574,
  0.03732297,
  0.24155128,
  0.0,
  0.09111779,
  0.027868494,
  0.0,
  0.56325394,
  0.032311752,
  0.0894183,
  0.07526479,
  0.5819565,
  0.03524498,
  0.052858017,
  0.9324601,
  0.5932759,
  0.10209701,
  0.0,
  0.0,
  0.0,
  0.06550356,
  0.10148247,
  0.074556716,
  0.57820094,
  0.0,
  0.28912574,
  0.025134742,
  0.09552961,
  0.29451066,
  1.0107119,
  0.0,
  0.0,
  0.26959148,
  0.1352905,
  0.9940389,
  0.5878116,
  0.06622862,
  0.14897093,
  0.047481902,
  0.717609,
  0.030159973,
  0.6569805,
  0.12385332,
  0.14970434,
 

In [89]:
 pd.DataFrame(zip(list(monthly_income_per_user[list(monthly_income_per_user.columns)[0]].iloc[test_index]), 
                       list(monthly_income_per_user.iloc[test_index][30]), prediction[0]))

,0,1,2
0,0.0,0.021826,0.743522
1,0.0,1.000000,1.038087
2,0.0,0.351125,0.499905
3,0.0,0.046095,0.105585
4,0.0,1.000000,0.000000
...,...,...,...
3749,0.0,0.153790,0.583577
3750,0.0,0.021195,0.035943
3751,0.0,0.038174,1.013254
3752,0.0,0.724867,0.197466


In [90]:
residuals =  pd.DataFrame(zip(list(monthly_income_per_user[list(monthly_income_per_user.columns)[0]].iloc[test_index]), 
                       list(monthly_income_per_user.iloc[test_index][30]), prediction[0])).rename(columns={0: "user_id", 1: "actual", 2: "prediction"})
display(residuals.head())
residuals['residuals'] = residuals.apply(lambda row: abs(row['actual'] - row['prediction']), axis=1) 
import plotly.express as px
fig = px.histogram(residuals, x="residuals")
fig.show()

,user_id,actual,prediction
0,0.0,0.021826,0.743522
1,0.0,1.000000,1.038087
2,0.0,0.351125,0.499905
3,0.0,0.046095,0.105585
4,0.0,1.000000,0.000000


### prediction

In [230]:
prediction_df = monthly_income_per_user[list(monthly_income_per_user.columns)[2:]]
display(prediction_df.head())

,5,6,7,8,9,10,11,12,13,14,...,24,25,26,27,28,29,30,pred_1,pred_2,pred_3
0,0.104777,0.217127,0.187735,0.147529,0.086186,0.035227,0.000617,0.126737,0.022186,0.083393,...,0.014364,0.006979,0.000000,0.344446,1.000000,0.136214,0.677449,0.180811,0.126776,0.085586
1,0.105283,0.703729,0.265429,0.484029,0.208140,0.026550,0.012020,0.131871,0.057360,0.243094,...,0.029523,0.187453,0.000000,1.000000,0.072121,0.480223,0.021826,0.071856,0.052594,0.052441
2,0.000000,0.034936,1.000000,0.095036,0.066250,0.187015,0.063853,0.142752,0.041786,0.023731,...,0.403631,0.119683,0.022831,0.005315,0.138356,0.084608,0.064432,0.053279,0.052918,0.052794
3,0.688249,0.014252,0.000000,0.356258,1.000000,0.422344,0.525300,0.166130,0.083322,0.324877,...,0.168836,0.186765,0.006628,0.052636,0.086610,0.053512,0.077222,0.053175,0.053013,0.052829
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.499443,0.502953,0.502515,1.000000,0.502001,0.503468,0.000000,0.155886,0.123781,0.092262


In [195]:
columns = list(monthly_income_per_user.columns)
num_cols = len(columns)
model_columns = columns[1:]
num_of_rows = len(monthly_income_per_user)

In [196]:
num_of_rows, len(model_columns)

(18766, 30)

In [197]:
_prediction = pd.DataFrame([i[0] for i in model.predict(monthly_income_per_user[model_columns].values.reshape(num_of_rows, len(model_columns), 1))]).rename(columns={0: "pred_1"})

In [198]:
monthly_income_per_user = pd.concat([monthly_income_per_user[model_columns[1:]].reset_index(drop=True), _prediction.reset_index(drop=True)], axis=1)

In [199]:
monthly_income_per_user.head()

,2,3,4,5,6,7,8,9,10,11,...,22,23,24,25,26,27,28,29,30,pred_1
0,0.489058,0.079057,0.712987,0.104777,0.217127,0.187735,0.147529,0.086186,0.035227,0.000617,...,0.012574,0.319365,0.014364,0.006979,0.000000,0.344446,1.000000,0.136214,0.677449,0.180811
1,0.000000,0.040489,0.239213,0.105283,0.703729,0.265429,0.484029,0.208140,0.026550,0.012020,...,0.094264,0.019401,0.029523,0.187453,0.000000,1.000000,0.072121,0.480223,0.021826,0.071856
2,0.080794,0.217862,0.601820,0.000000,0.034936,1.000000,0.095036,0.066250,0.187015,0.063853,...,0.021275,0.257200,0.403631,0.119683,0.022831,0.005315,0.138356,0.084608,0.064432,0.053279
3,0.045426,0.005009,0.136179,0.688249,0.014252,0.000000,0.356258,1.000000,0.422344,0.525300,...,0.357498,0.025642,0.168836,0.186765,0.006628,0.052636,0.086610,0.053512,0.077222,0.053175
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.037964,0.818332,0.499443,0.502953,0.502515,1.000000,0.502001,0.503468,0.000000,0.155886


In [200]:
_prediction = pd.DataFrame([i[0] for i in 
                            model.predict(monthly_income_per_user.values.reshape(num_of_rows, len(model_columns), 1))
                           ]).rename(columns={0: "pred_2"})
_prediction.head()

,pred_2
0,0.126776
1,0.052594
2,0.052918
3,0.053013
4,0.123781


In [208]:
monthly_income_per_user = pd.concat([monthly_income_per_user[model_columns[2:] + ['pred_1']].reset_index(drop=True), _prediction.reset_index(drop=True)], axis=1)

In [221]:
monthly_income_per_user[model_columns[3:] + ['pred_1', 'pred_2']]

,4,5,6,7,8,9,10,11,12,13,...,23,24,25,26,27,28,29,30,pred_1,pred_2
0,0.712987,0.104777,0.217127,0.187735,0.147529,0.086186,0.035227,0.000617,0.126737,0.022186,...,0.319365,0.014364,0.006979,0.000000,0.344446,1.000000,0.136214,0.677449,0.180811,0.126776
1,0.239213,0.105283,0.703729,0.265429,0.484029,0.208140,0.026550,0.012020,0.131871,0.057360,...,0.019401,0.029523,0.187453,0.000000,1.000000,0.072121,0.480223,0.021826,0.071856,0.052594
2,0.601820,0.000000,0.034936,1.000000,0.095036,0.066250,0.187015,0.063853,0.142752,0.041786,...,0.257200,0.403631,0.119683,0.022831,0.005315,0.138356,0.084608,0.064432,0.053279,0.052918
3,0.136179,0.688249,0.014252,0.000000,0.356258,1.000000,0.422344,0.525300,0.166130,0.083322,...,0.025642,0.168836,0.186765,0.006628,0.052636,0.086610,0.053512,0.077222,0.053175,0.053013
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.818332,0.499443,0.502953,0.502515,1.000000,0.502001,0.503468,0.000000,0.155886,0.123781
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18761,0.040057,0.128427,0.288363,0.487666,0.200875,0.034228,0.391209,0.096246,0.238660,0.091901,...,0.000000,1.000000,0.000000,0.412083,0.304416,0.081222,0.081638,0.190102,0.063750,0.053766
18762,0.001611,0.174307,0.007942,0.004247,0.182322,0.161888,0.100126,0.257190,0.225872,0.201260,...,0.240750,0.034221,0.161888,0.151874,0.206219,0.016223,0.134840,0.000000,0.052589,0.044298
18763,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.347315,0.492577,...,0.666835,0.716888,0.096778,0.000000,0.032344,0.564624,0.376425,0.913834,0.270642,0.177569
18764,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.717770,0.000000,0.038680,0.047791


In [214]:
_prediction = pd.DataFrame([i[0] for i in 
                            model.predict(monthly_income_per_user[model_columns[2:] + ['pred_1', 'pred_2']].values.reshape(num_of_rows, len(model_columns), 1))
                           ]).rename(columns={0: "pred_3"})

In [224]:
monthly_income_per_user = pd.concat([monthly_income_per_user.reset_index(drop=True), _prediction.reset_index(drop=True)], axis=1)

In [225]:
monthly_income_per_user.head()

,3,4,5,6,7,8,9,10,11,12,...,24,25,26,27,28,29,30,pred_1,pred_2,pred_3
0,0.079057,0.712987,0.104777,0.217127,0.187735,0.147529,0.086186,0.035227,0.000617,0.126737,...,0.014364,0.006979,0.000000,0.344446,1.000000,0.136214,0.677449,0.180811,0.126776,0.085586
1,0.040489,0.239213,0.105283,0.703729,0.265429,0.484029,0.208140,0.026550,0.012020,0.131871,...,0.029523,0.187453,0.000000,1.000000,0.072121,0.480223,0.021826,0.071856,0.052594,0.052441
2,0.217862,0.601820,0.000000,0.034936,1.000000,0.095036,0.066250,0.187015,0.063853,0.142752,...,0.403631,0.119683,0.022831,0.005315,0.138356,0.084608,0.064432,0.053279,0.052918,0.052794
3,0.005009,0.136179,0.688249,0.014252,0.000000,0.356258,1.000000,0.422344,0.525300,0.166130,...,0.168836,0.186765,0.006628,0.052636,0.086610,0.053512,0.077222,0.053175,0.053013,0.052829
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.499443,0.502953,0.502515,1.000000,0.502001,0.503468,0.000000,0.155886,0.123781,0.092262


In [227]:
monthly_income_per_user = pd.merge(monthly_income_per_user, )

18766

In [226]:
prediction_df.to_csv("monetary_value_for_clv_calculation.csv")